In [11]:
#since I am using colab.research.google.com have to do the follwoing to import my data set
from google.colab import files

!rm Audiobooks-data.csv
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Audiobooks-data.csv to Audiobooks-data.csv
User uploaded file "Audiobooks-data.csv" with length 728020 bytes


In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn import preprocessing
import matplotlib.pyplot as plt

In [13]:
#read raw data from file
raw_data = pd.read_csv('Audiobooks-data.csv')
raw_data.head()

,ID,Book Length (mins) overall,Book Length avg,Price overall,Price Avg,Review,Review 10/10,Minutes listened,Completion,Support Requests,Last time visited purchase data,Targets
0,994,1620.0,1620,19.73,19.73,1,10.00,0.99,1603.8,5,92,0
1,1143,2160.0,2160,5.33,5.33,0,8.91,0.00,0.0,0,0,0
2,2059,2160.0,2160,5.33,5.33,0,8.91,0.00,0.0,0,388,0
3,2882,1620.0,1620,5.96,5.96,0,8.91,0.42,680.4,1,129,0
4,3342,2160.0,2160,5.33,5.33,0,8.91,0.22,475.2,0,361,0


In [14]:
#first column is id and last is target so we can ignore these for now
unscaled_raw_data = raw_data.iloc[:,1:-1]
unscaled_raw_data.head()
#unscaled_raw_data.shape

,Book Length (mins) overall,Book Length avg,Price overall,Price Avg,Review,Review 10/10,Minutes listened,Completion,Support Requests,Last time visited purchase data
0,1620.0,1620,19.73,19.73,1,10.00,0.99,1603.8,5,92
1,2160.0,2160,5.33,5.33,0,8.91,0.00,0.0,0,0
2,2160.0,2160,5.33,5.33,0,8.91,0.00,0.0,0,388
3,1620.0,1620,5.96,5.96,0,8.91,0.42,680.4,1,129
4,2160.0,2160,5.33,5.33,0,8.91,0.22,475.2,0,361


In [15]:
#storing targets
targets_all = raw_data.iloc[:,-1]
targets_all.head()

0    0
1    0
2    0
3    0
4    0
Name:  Targets, dtype: int64

In [0]:
#balance data set as there are more 0s in targets than 1s
#so count 1s so that equal 0s can be used

target_one_count = int(np.sum(targets_all)) #as we have 0s and 1s in target so sum of all targets will give us count of 1s
#now traverse over data set and select indices to keep and the ones to delete.Start deleting when count for 0s becomes equal to count of 1s
target_zero_count = 0
indices_to_delete_for_balancing = []

no_of_rows_in_data_set = unscaled_raw_data.shape[0] #gets us the number of rows in data set

for i in range(no_of_rows_in_data_set):
  if(targets_all[i] == 0):
    target_zero_count += 1
    if(target_zero_count > target_one_count):
      indices_to_delete_for_balancing.append(i)
      
unscaled_but_balanced_data = unscaled_raw_data.drop(indices_to_delete_for_balancing, axis=0) #axis 0 as we want to delete rows
balanced_targets = targets_all.drop(indices_to_delete_for_balancing,axis=0)

In [17]:
#now that we have balanced data.
#we shall then scale/standardize the data

scaled_and_balanced_data = preprocessing.scale(unscaled_but_balanced_data) #stadardizes all inputs. each column is treated as an array and then standardized

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.
  


In [0]:
#now lets shuffle the input data
shuffled_indices = np.arange(scaled_and_balanced_data.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_scaled_and_balanced_data = scaled_and_balanced_data[shuffled_indices]
shuffled_balanced_targets = balanced_targets.values[shuffled_indices]

In [19]:
#split into training, validation and test
sample_count_of_preprocess_data = shuffled_scaled_and_balanced_data.shape[0]

training_sample_count = int(0.8*sample_count_of_preprocess_data)
validation_sample_count = int(0.1*sample_count_of_preprocess_data)
test_sample_count = sample_count_of_preprocess_data - training_sample_count - validation_sample_count

#time to divide data into these training, validation and test
training_data = shuffled_scaled_and_balanced_data[:training_sample_count] #start to training_sample_count
validation_data = shuffled_scaled_and_balanced_data[training_sample_count : training_sample_count + validation_sample_count]
test_data = shuffled_scaled_and_balanced_data[training_sample_count + validation_sample_count : ] #everything that is left

training_targets = shuffled_balanced_targets[:training_sample_count]
validation_targets = shuffled_balanced_targets[training_sample_count : training_sample_count + validation_sample_count]
test_targets = shuffled_balanced_targets[training_sample_count + validation_sample_count :]

#since we have balanced the data before spliting, it is a good idea to check here if the splitted data are balanced too. should be close to 50% for each seta s we only have 2 categories
print("% of 1s in training data: " + str(np.sum(training_targets)/training_sample_count))
print("% of 1s in validation data: " + str(np.sum(validation_targets)/validation_sample_count))
print("% of 1s in test data: " + str(np.sum(test_targets)/test_sample_count))

#also that the total count adds up
print("total: " + str(training_sample_count + validation_sample_count + test_sample_count))

% of 1s in training data: 0.5037720033528919
% of 1s in validation data: 0.48769574944071586
% of 1s in test data: 0.48214285714285715
total: 4474


In [0]:
#save 3 data sets in npz files
np.savez('Audiobook_Training_Data.npz',inputs=training_data, targets = training_targets)
np.savez('Audiobook_Validation_Data.npz',inputs=validation_data, targets = validation_targets)
np.savez('Audiobook_Test_Data.npz',inputs=test_data, targets = test_targets)
#data now is preprocessed. So pre processing complete